# Testing environment for DoSE

## Setup

### Load libraries

In [1]:
import pandas as pd
import numpy as np 
import gseapy
from biothings_client import get_client

### Define data

In [2]:
seeds_file = "Input/0007079.txt"
betweenness_file = "Input/0007079_added_200_dmd_betweenness_hub_0.01.txt"
significance_file = "Input/0007079_added_200_dmd_significance_hub_1.txt"
diseases_file = "Input/ICD10_commROCG_raw.txt"
disease_clusters_file = "Input/ICD10_commROCG_cluster.txt"

### Load data

In [3]:
seeds = pd.read_csv(seeds_file, sep="\t", header=None)[0]
betweenness = pd.read_csv(betweenness_file, sep="\t")['node']
significance = pd.read_csv(significance_file, sep="\t")['node']
diseases = pd.read_csv(diseases_file, sep="\t", header=None)
disease_clusters = pd.read_csv(disease_clusters_file, sep="\t", header=None)

In [4]:
import timeit
start = timeit.default_timer()

stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  4.1903999999703956e-05


## Mapper

In [5]:
id_type_key = {'entrez':'entrezgene','ensembl':'ensembl.gene','symbol':'symbol','uniprot':'uniprot.Swiss-Prot','mondo':'mondo'}
gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']

In [6]:
def preprocess_results(mapping, multicol, singlecol, key, explode=False):
    
    def convert_to_string(cell, key):
        if str(cell) != 'nan':
            extracted_ids = [val.get(key) for val in cell]
            return ';'.join(str(e) for e in list(set(extracted_ids)))
        return cell
    mapping[multicol] = mapping[multicol].apply(lambda x: convert_to_string(x, key)) if multicol in mapping else np.nan
    if singlecol in mapping:
        mapping[multicol].fillna(mapping[singlecol], inplace=True)
        mapping = mapping.drop(columns=[singlecol])
    if explode:
        mapping = mapping[multicol].split(';').explode(multicol)
        mapping.rename(columns={multicol: singlecol}, inplace = True)
    return mapping


def get_prev_mapping(in_set, id_type, file, sep):
    # ===== Get mapping from local mapping file =====
    mapping = pd.read_csv(file, sep=sep, header=0, dtype=str)
    if id_type == "ICD-10":
        mapping = split_and_expand_column(data=mapping, split_string=",", column_name="ICD-10")
        mapping_copy = mapping.copy()
        mapping_copy['ICD-10'] = mapping_copy['ICD-10'].str.split('.', expand=True)[0]
        mapping = pd.concat([mapping, mapping_copy], ignore_index=True)
    # ==== Map given disease set ====
    id_type = id_type_key[id_type] if id_type in id_type_key else id_type
    mapped_set = mapping[mapping[id_type].isin(in_set)]
    # ===== Get missing values =====
    missing = list(set(in_set) - set(mapping[id_type]))
    return mapped_set, missing, mapping
    

def get_gene_mapping(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    # ===== Get mapping for missing values =====
    if len(missing) > 0:
        mg = get_client("gene")
        mapping = mg.querymany(missing, scopes=id_type_key[id_type], fields=','.join(gene_ids),
                     species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping = mapping.drop(columns=[id_type_key[id_type]])
        mapping.rename(columns={'query': id_type_key[id_type]}, inplace = True)
        # ===== Split if there are multiple ensembl ids =====
        if 'ensembl' in mapping:
            mapping = preprocess_results(mapping=mapping, multicol='ensembl', singlecol='ensembl.gene', key='gene', explode=True)
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_id_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    return df

def get_gene_to_attributes(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get gene ID mappings =====
    gene_mapping, _, _ = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    df, missing, prev_mapping = get_prev_mapping(in_set=set(gene_mapping['entrezgene']), id_type='entrez', file='gene_att_mapping.csv', sep=",")
    if len(missing) > 0:
        mg = get_client("gene")
        gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']
        mapping = mg.querymany(missing, scopes=','.join(gene_ids),
                            fields='pathway.kegg.id, go.BP.id, go.CC.id, go.MF.id',
                            species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'entrezgene'}, inplace = True)
        for column in ['go.BP','go.CC','go.MF','pathway.kegg']:
            mapping = preprocess_results(mapping=mapping, multicol=column, singlecol=column+'.id', key='id')
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_att_mapping.csv', index=False)        
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # work with not unique values...
    mapping_subset = gene_mapping[['entrezgene', id_type_key[id_type]]].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['entrezgene'], how = 'outer')
    df = df.drop(columns=['entrezgene'])
    df = df.fillna('').groupby([id_type_key[id_type]], as_index=False).agg({'go.BP': combine_rows, 'go.CC': combine_rows,
                                                                            'go.MF': combine_rows, 'pathway.kegg': combine_rows})
    return df

In [8]:
start = timeit.default_timer()
reference_mapping = get_gene_mapping(seeds, 'uniprot')
target_mapping = get_gene_mapping(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.0206533390000061


In [9]:
reference_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
191,125,ENSG00000196616,ADH1B,P00325
192,2555,ENSG00000151834,GABRA2,P47869
193,126,ENSG00000248144,ADH1C,P00326
194,3356,ENSG00000102468,HTR2A,P28223


In [10]:
target_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
0,1394,ENSG00000120088,CRHR1,P34998
1,1394,ENSG00000276191,CRHR1,P34998
2,104909134,ENSG00000263715,LINC02210-CRHR1,P34998
3,104909134,ENSG00000278232,LINC02210-CRHR1,P34998
4,104909134,ENSG00000282456,LINC02210-CRHR1,P34998
...,...,...,...,...
212,1742,ENSG00000132535,DLG4,P78352
213,4684,ENSG00000149294,NCAM1,P13591
214,2778,ENSG00000087460,GNAS,O95467
215,2776,ENSG00000156052,GNAQ,P50148


In [11]:
start = timeit.default_timer()
reference_kegg_mapping = get_gene_to_attributes(seeds, 'uniprot')
target_kegg_mapping = get_gene_to_attributes(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.05625762200000395


In [12]:
reference_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,P00325,"{GO:0042572, GO:0001523, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
1,P00326,"{GO:0042572, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004022, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
2,P28223,"{GO:0006874, GO:0046718, GO:0030431, GO:000821...","{GO:0043025, GO:0030425, GO:0005886, GO:000588...","{GO:0044877, GO:0030594, GO:0051378, GO:000196...","{hsa04726, hsa04020, hsa04080, hsa04540, hsa04..."
3,P47869,"{GO:0060078, GO:1904862, GO:0051932, GO:000726...","{GO:0098982, GO:0043025, GO:0032590, GO:000588...","{GO:0030594, GO:0004890, GO:0005254, GO:002285...","{hsa05032, hsa04727, hsa04080, hsa04723, hsa04..."


In [13]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0015031, GO:0048010, GO:0042307, GO:004685...","{GO:0005634, GO:0005829, GO:0005942}","{GO:0001784, GO:0046982, GO:0030971, GO:001990...","{hsa04072, hsa04662, hsa05169, hsa04722, hsa05..."
1,O14492,"{GO:0050873, GO:0001922, GO:0007596, GO:005085...","{GO:0001725, GO:0005886, GO:0005737, GO:000588...","{GO:0005515, GO:0005068, GO:0042169, GO:004280...","{hsa04910, hsa04722}"
2,O14610,"{GO:0007602, GO:0007186}",{GO:0005834},"{GO:0003924, GO:0031681}","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
3,O14775,"{GO:0043547, GO:1901386, GO:0007186, GO:000645...","{GO:0098793, GO:0005737, GO:0005634, GO:190277...","{GO:0003924, GO:0031682, GO:0030159, GO:000551...","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
4,O14842,"{GO:0042593, GO:0032024, GO:0051928, GO:003007...","{GO:0005887, GO:0005886}","{GO:0008289, GO:0045125, GO:0004930}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0016339, GO:0050808, GO:0007156, GO:0007155}","{GO:0016020, GO:0005887}",{GO:0005509},{}
196,Q9UNN8,"{GO:0007596, GO:0050819}","{GO:0005886, GO:0005615, GO:0005887, GO:000557...","{GO:0038023, GO:0005515}",{hsa04610}
197,Q9UQC2,"{GO:0048015, GO:0007169, GO:0043306, GO:000741...","{GO:0005737, GO:0005886, GO:0005829}","{GO:0043325, GO:0005068, GO:0005547, GO:0005515}","{hsa04072, hsa04071, hsa04014, hsa04666, hsa04..."
198,Q9Y2G0,"{GO:0046854, GO:0072659}","{GO:0015629, GO:0005886, GO:0005829}",{GO:0005515},{}


In [19]:
full_ids_mapping = pd.read_csv("../disorders.map", sep="\t", dtype=str)
full_ids_mapping

,mondo,omim,snomedct,umls,orpha,mesh,ncit,doid,meddra,medgen,ICD-10
0,0008118,164330,716180009,C1834013,2724,C537740,NaN,NaN,NaN,NaN,NaN
1,0010439,300829,NaN,C1853577,NaN,C543241,NaN,NaN,NaN,NaN,NaN
2,0008117,164310,763829004,C1834014,98897,C563508,NaN,NaN,NaN,NaN,G71.0
3,0009448,242600,84121007,C0268654,42062,C536285,NaN,NaN,NaN,NaN,E72.0
4,0008119,164400,715748006,C0752120,98755,NaN,C129982,0050954,NaN,NaN,G11.8
...,...,...,...,...,...,...,...,...,...,...,...
24115,0009507,245550,732961003,C1855551,1296,C538396,NaN,NaN,NaN,NaN,Q87.8
24116,0009508,245552,NaN,C1855550,NaN,C537549,NaN,NaN,NaN,NaN,NaN
24117,0009501,245340,766715000,C1855577,171690,C565449,NaN,NaN,NaN,NaN,G72.8
24118,0009502,245348,NaN,C1855565,79244,C565448,NaN,NaN,NaN,NaN,E74.4


In [20]:
with np.printoptions(threshold=np.inf):
    print(full_ids_mapping['ICD-10'].unique())

[nan 'G71.0' 'E72.0' 'G11.8' 'Q56.1' 'H90.3' 'G31.8' 'Q80.4' 'Q87.0'
 'Q80.2' 'Q87.8' 'F42' 'Q77.7' 'D64.4' 'E85.0' 'G12.2' 'E77.8' 'E03.1'
 'G30' 'H53.8' 'Q77.3' 'E83.3' 'Q83.3' 'G11.4' 'D81.8' 'Q04.3' 'H35.5'
 'C41.9' 'E70.3' 'D64.0' 'D18.0' 'Q82.8,Q43.8' 'H47.2' 'G93.2' 'H47.0'
 'K83.1' 'K59.8' 'N46' 'E80.0' 'D80.0' 'E27.4' 'Q99.8' 'E76.0' 'G60.8'
 'H18.5' 'D84.8' 'Q78.8' 'D81.4' 'C49.9' 'Q87.3' 'G24.5' 'Q67.4' 'G93.4'
 'E80.5' 'E72.5' 'E88.8' 'E74.3' 'M06.8' 'F72' 'H90.5' 'G11.0' 'D80.5'
 'D44.8' 'J84.81' 'G23.0' 'Q87.1' 'G51.4' 'I42.0' 'E28.3,E29.1' 'E26.8'
 'Q71.6' 'Q75.4' 'E29.1' 'E72.8' 'Q34.8' 'E27.1' 'C26.9' 'E74.0' 'E31.0'
 'E54' 'Q74.0' 'S00.T98' 'G71.1' 'E78.5' 'J39.2' 'Q45.1' 'Q10.3' 'K86.1'
 'E67.1' 'M60,G72.49,M60.9' 'G60.0' 'M61.5' 'Q78.2' 'Q82.8' 'Q11.2'
 'T78.3' 'E23.0' 'G20' 'E71.1,E71.110' 'Q41.9,Q41.8,Q41.2,Q41.1,Q41.0'
 'M93.2' 'Q78.4' 'Q78.0' 'N00.N99' 'D81.1' 'E22.0' 'Q77.8' 'C44.5' 'J96.0'
 'C56' 'Q02' 'N20,N20.2' 'G40.4' 'D58.1' 'D27' 'E75.2,E75.23' 'N97'
 'K

In [28]:
import re
changes = list()
for idx, row in full_ids_mapping.iterrows():
    if  not (isinstance(full_ids_mapping.loc[idx, 'ICD-10'], float) and np.isnan(full_ids_mapping.loc[idx, 'ICD-10'])):
        new_ids = set()
        cur_ids = full_ids_mapping.loc[idx, 'ICD-10'].split(",")
        for cur_id in cur_ids:
            ids = list()
            if "-" in cur_id:
                ids = cur_id.split("-")
                print(cur_id)
            elif re.search(r'[A-Z][0-9]{2}[.][A-Z][0-9]{2}', cur_id):
                ids = cur_id.split(".")
            else:
                ids = re.findall(r"([A-Z][0-9]{1,})[.,-]?", cur_id)
            changes.append(cur_id+" ----> "+str(ids))
            new_ids.update(ids)
    print(new_ids)

        


{'E74'}
{'E74'}
{'G71'}
{'E72'}
{'G11'}
{'G11'}
{'G11'}
{'Q56'}
{'H90'}
{'H90'}
{'G31'}
{'G31'}
{'G31'}
{'G31'}
{'G31'}
{'G31'}
{'Q80'}
{'Q87'}
{'Q87'}
{'Q80'}
{'Q87'}
{'Q87'}
{'F42'}
{'F42'}
{'F42'}
{'F42'}
{'Q87'}
{'G71'}
{'G71'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q77'}
{'H90'}
{'H90'}
{'H90'}
{'D64'}
{'E85'}
{'E85'}
{'E85'}
{'G12'}
{'G12'}
{'E77'}
{'E77'}
{'E77'}
{'Q80'}
{'Q80'}
{'Q80'}
{'Q80'}
{'Q80'}
{'E03'}
{'Q87'}
{'Q87'}
{'G30'}
{'H53'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'E83'}
{'E83'}
{'E83'}
{'E83'}
{'Q83'}
{'Q83'}
{'Q83'}
{'Q83'}
{'Q83'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'G11'}
{'G11'}
{'G11'}
{'D81'}
{'G71'}
{'G71'}
{'G71'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'E77'}
{'E77'}
{'Q04'}
{'H35'}
{'C41'}
{'G11'}
{'E70'}
{'E70'}
{'D64'}
{'D64'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'D18'}
{'Q82', 'Q43'}
{

{'C17'}
A00-A09
{'A00', 'A09'}
{'A00', 'A09'}
{'F01'}
{'I42'}
{'H90'}
{'H35'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'C24'}
{'D80'}
{'Q87'}
{'Q78'}
{'G11'}
{'G11'}
{'G11'}
{'Q87'}
{'D47'}
{'D47'}
{'D47'}
{'D84'}
{'D84'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'R82'}
{'R82'}
{'E83'}
{'E83'}
{'E83'}
{'E83'}
{'G11'}
{'Q87'}
{'L73'}
{'L73'}
{'Q93'}
{'G60'}
{'Q87'}
{'Q87'}
{'Q78'}
{'Q78'}
{'G71'}
{'Q82'}
{'E79'}
{'G11'}
{'Q71'}
{'E75'}
{'E75'}
{'E75'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'D68'}
{'D68'}
{'D68'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q79'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'G11'}
{'G11'}
{'Q82'}
{'Q82'}
{'Q82'}
{'G40'}
{'G40'}
{'G40'}
{'G40'}
{'Q87'}
{'Q87'}
{'Q87'}
{'D84'}
{'

{'Q77'}
{'Q77'}
{'G12'}
{'G12'}
{'G12'}
set()
set()
set()
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'H66'}
{'I78'}
{'G62'}
{'G62'}
{'I82'}
{'I82'}
{'I82'}
{'B64', 'A07', 'B50'}
{'B64', 'A07', 'B50'}
{'Q93'}
{'Q93'}
{'Q93'}
{'D84'}
{'D84'}
{'D84'}
{'E75'}
{'E75'}
{'G24'}
{'G60'}
{'G60'}
{'D84'}
{'H35'}
{'D84'}
{'E71'}
{'E71'}
{'E71'}
{'D84'}
{'Q61'}
{'Q61'}
{'C71'}
{'C71'}
{'C71'}
{'C71'}
{'C71'}
{'C71'}
{'H18'}
{'H18'}
{'H18'}
{'H18'}
{'H18'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'E06'}
{'G21'}
{'G21'}
{'E28'}
{'E28'}
{'B52'}
{'B52'}
{'B52'}
{'B52'}
{'B52'}
{'H90'}
{'G30'}
{'G30'}
{'G11'}
{'E20'}
{'E20'}
{'E20'}
{'E10'}
{'N04'}
{'N04'}
{'N04'}
{'G30'}
{'E23'}
{'E23'}
{'E23'}
{'E23'}
{'H90'}
{'G24'}
{'G24'}
{'Q87'}
{'E71'}
{'Q78'}
{'Q78'}
{'H34'}
{'H34'}
{'H57'}
{'H18'}
{'H18'}
{'B37'}
{'B37'}
{'B37'}
{'B37'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'K91'}
{'D28'}
{'D28'}
{'D28'}
{'D28'}
{'

{'A77'}
{'A77'}
{'A77'}
{'A77'}
{'A77'}
{'H90'}
{'H90'}
{'H90'}
{'H90'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q12'}
{'E10'}
{'E10'}
{'Q82'}
{'Q93'}
{'Q93'}
{'Q93'}
{'Q77'}
{'Q77'}
{'Q87'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'E10'}
{'N36'}
{'N36'}
{'N36'}
{'H15'}
{'M73', 'A54'}
{'H18'}
{'H18'}
{'H18'}
{'I65'}
{'I65'}
{'D61'}
{'D61'}
{'D61'}
{'G60'}
{'G60'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'Q77'}
{'K83'}
{'K72'}
{'K72'}
{'D86'}
{'D86'}
{'D86'}
{'H53'}
{'H53'}
{'A22'}
{'A22'}
{'D84'}
{'D84'}
{'D84'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'E21'}
{'E21'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'H40'}
{'P39'}
{'P39'}
{'A33'}
{'Q85'}
{'Q85'}
{'Q85'}
{'Q85'}
{'N30'}
{'N30'}
{'N30'}
{'N30'}
{'N34'}
{'N34'}
{'N34'}
{'N34'}
{'N34'}
{'N34'}
{'N34'}
{'F84'}
{'F84'}
{'C32'}
{'C32'}
{'K90'}
{'D53'}
{'D53'}
{'I07'}
{'Q68'}
{'Q68'}
{'Q68'}
{'Q68'}
{'Q68'}
{'Q68'}
{'G24'}
{'G24'}
{'G24'}
{'G24'}
{'G24'}
{'G24'}
{'G24'}
{'G24'}
{

{'D81'}
{'D81'}
{'D81'}
{'D81'}
{'D81'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'F64'}
{'C74'}
{'C74'}
{'I40'}
{'I40'}
{'I40'}
{'I40'}
{'I40'}
{'I40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'C43', 'C44'}
{'F80'}
{'A26'}
{'H53'}
{'H53'}
{'H53'}
{'E83'}
{'E83'}
{'E83'}
{'H93'}
{'H93'}
{'H93'}
{'H93'}
{'H53'}
{'H53'}
{'H53'}
{'H53'}
{'K31'}
{'K31'}
{'Q40'}
{'Q40'}
{'Q40'}
{'Q40'}
{'G71'}
{'G71'}
{'Q60'}
{'Q60'}
{'N81'}
{'N81'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'C69'}
{'E77'}
{'E77'}
{'E77'}
{'E70'}
{'E70'}
{'E70'}
{'G11'}
{'E77'}
{'D84'}
{'D84'}
{'D84'}
{'D84'}
{'E70'}
{'E70'}
{'E70'}
{'E70'}
{'E70'}
{'E70'}
{'M70'}
{'E83'}
{'E83'}
{'E83'}
{'E83'}
{'L08'}
{'L08'}
{'P72'}
{'P72'}
{'P72'}
{'L60'}
{'L60'}
{'L60'}
{'L60'}
{'L60'}
{'L60'}
{'N77', 'N76'}
{'N77', 'N76'}
{'N77', 'N76'}
{'N77', 'N76'}
{'N77', 'N76'

...

{'Q87'}
{'Q87'}
{'C15'}
{'C15'}
{'D47'}
{'D47'}
{'D47'}
{'D47'}
{'D47'}
{'D47'}
{'E04'}
{'E04'}
{'E04'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'K63'}
{'K63'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'D69'}
{'I33'}
{'I33'}
{'I33'}
{'I33'}
{'Q30'}
{'G24'}
{'L50'}
{'K22'}
{'K22'}
{'K22'}
{'D71'}
{'Q60'}
{'Q60'}
{'Q60'}
{'E70'}
{'E70'}
{'E70'}
{'E70'}
{'Q61'}
{'G23'}
{'G23'}
{'D13'}
{'Q63'}
{'Q63'}
{'Q43'}
{'Q43'}
{'Q43'}
{'Q43'}
{'Q43'}
{'Q43'}
{'Q10'}
{'Q14'}
{'Q14'}
{'Q14'}
{'A69'}
{'N30'}
{'A68'}
{'L67'}
{'L67'}
{'L67'}
E08-E13
{'E08', 'E10', 'E13', 'E14', 'E11'}
{'E08', 'E10', 'E13', 'E14', 'E11'}
{'E08', 'E10', 'E13', 'E14', 'E11'}
{'C25'}
{'C25'}
{'C25'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q27'}
{'Q27'}
I20-I25
{'I25', 'K76', 'I20'}
{'N07'}
{'N07'}
{'N07'}
{'N07'}
{'N07'}
{'K50'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'C43'}
{'G40'}
{'Q61'

{'C96'}
{'C96'}
{'E75'}
{'E75'}
{'E75'}
{'C96'}
{'Q96'}
{'Q96'}
{'Q77'}
{'Q77'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q98'}
{'Q96'}
{'Q99'}
{'Q99'}
{'Q99'}
{'D20'}
{'D20'}
{'D20'}
{'H30'}
{'D76'}
{'G40'}
{'Q74'}
{'G40'}
{'G12'}
{'G40'}
{'Q93'}
{'G40'}
{'G40'}
{'E88'}
{'G71'}
{'Q82'}
{'D58'}
{'D58'}
{'D58'}
{'D58'}
{'D58'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q26'}
{'Q26'}
{'Q26'}
{'Q26'}
{'Q00'}
{'Q00'}
{'D17'}
{'D17'}
{'K00'}
{'K00'}
{'Q26'}
{'Q26'}
{'Q84'}
{'Q26'}
{'Q10'}
{'Q10'}
{'Q10'}
{'Q10'}
{'Q26'}
{'G11'}
{'Q26'}
{'Q26'}
{'Q26'}
{'Q87'}
{'Q21'}
{'C94'}
{'Q93'}
{'M89'}
{'Q21'}
{'Q21'}
{'G11'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q04'}
{'D47'}
{'D47'}
{'D47'}
{'D47'}
{'D46'}
{'Q87'}
{'Q04'}
{'D46'}
{'C94'}
{'M83'}
{'C92'}
{'Q04'}
{'G40'}
{'C92'}
{'C92'}
{'C92'}
{'C92'}
{'C92'}
{'C92'}
{'D50'}
{'D50'}
{'D50'}
{'D50'}
{'C95'}
{'E23'}
{'E23'}
{'Q26'}
{'E78'}
{'Q26'}
{'Q26'}
{'Q26'}
{'G12'}
{'G12'}
{'G12'}


{'Q74'}
{'Q90'}
{'Q33'}
{'Q33'}
{'Q33'}
{'Q74'}
{'Q74'}
{'Q74'}
{'Q25'}
{'Q25'}
{'Q25'}
{'Q22'}
{'Q22'}
{'E70'}
{'L65'}
{'P28'}
{'P28'}
{'E77'}
{'E77'}
{'E77'}
{'E77'}
{'E77'}
{'E77'}
{'H35'}
{'H35'}
{'D56'}
{'D56'}
{'D56'}
{'D56'}
{'L50'}
{'L50'}
{'L50'}
{'L50'}
{'L50'}
{'L50'}
{'L50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'G50'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'I42'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q12'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'Q82'}
{'E79'}
{'E75'}
{'E75'}
{'E83'}
{'E70'}
{'E70'}
{'Q87'}
{'Q87'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'E75'}
{'Q68'}
{'Q68'}
{'Q68'}
{'O14'}
{'Q87'}
{'Q87'}
{'H47'}
C50-C50
{'C50'}
{'Q82'}
{'Q82'}
{'Q82'}
{'D81'}
{'D81'}
{'D81'}
{'D81'}
{'H49'}
{'D81'}
{'C22'}
{'G25'}
{'G25'}
{'G25'}
{'G25'}
{'G25'}
{'G25'}
{'G25'}
{'G25'}
{'Q39'}
{'Q39'}
{'Q39'}
{'Q39'}
{'J98'}
{'J98'}
{'J98'}
{'Q14'}
{'H35'}
{'H35'}
{'H35'}
{'H35'}


{'G71'}
{'Q87'}
{'E34'}
{'Q14'}
{'G71'}
{'F84'}
{'F84'}
{'F84'}
{'G71'}
{'Q82'}
{'Q82'}
{'Q82'}
{'E03'}
{'Q14'}
{'D15'}
{'H35'}
{'Q87'}
{'Q87'}
{'G71', 'Q78'}
{'Q87'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'Q87'}
{'Q82'}
{'Q82'}
{'Q82'}
{'G60'}
{'G60'}
{'E75'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'G60'}
{'Q87'}
{'Q87'}
{'B65'}
{'E75'}
{'E75'}
{'M42'}
{'M42'}
{'Q71'}
{'L81'}
{'Q87'}
{'Q87'}
{'E72'}
{'E77'}
{'G71'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q80'}
{'Q80'}
{'G71'}
{'G71'}
{'E77'}
{'G12'}
{'G12'}
{'G23'}
{'G12'}
{'Q87'}
{'N04'}
{'D11'}
{'D11'}
{'Q87'}
{'Q87'}
{'Q87'}
{'G20'}
{'G20'}
{'Q87'}
{'Q87'}
{'Q87'}
{'E71'}
{'E71'}
{'E71'}
{'E71'}
{'E71'}
{'E71'}
{'G71'}
{'Q87'}
{'Q87'}
{'M75'}
{'M75'}
{'M75'}
{'M75'}
{'G71'}
{'G71'}
{'G71'}
{'O26'}
{'O26'}
{'O26'}
{'G71'}
{'G71'}
{'G71'}
{'E51'}
{'E51'}
{'E51'}
{'E51'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'G71'}
{'Q87'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'G70'}
{'K75'}
{'K75'}
{'K75'}
{'K75'}
{'K75'}
{'K75'}
{

{'Q04'}
{'Q04'}
{'Q82'}
{'E72'}
{'E72'}
{'E72'}
{'G72'}
{'G72'}
{'E83'}
{'E83'}
{'E83'}
{'E83'}
{'E83'}
{'E72'}
{'E72'}
{'E72'}
{'E72'}
{'K05'}
{'Q87'}
{'Q87'}
{'Q87'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'G72'}
{'Q23', 'Q87'}
{'Q23', 'Q87'}
{'H90', 'H91'}
{'H90', 'H91'}
{'Q87'}
{'Q87'}
{'E83'}
{'E83'}
{'Q87'}
{'Q81'}
{'Q75'}
{'Q75'}
{'G11'}
{'L98'}
{'L98'}
{'L98'}
{'Q68'}
{'Q68'}
{'Q87'}
{'E72'}
{'E72'}
{'E72'}
{'E72'}
{'E72'}
{'E72'}
{'E72'}
{'Q87'}
{'Q78'}
{'Q78'}
{'J93'}
{'J93'}
{'J93'}
{'J93'}
{'Q87'}
{'L90'}
{'Q87'}
{'E23'}
{'L44'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'M85'}
{'E75'}
{'E70'}
{'I42'}
{'E78', 'E71'}
{'E78', 'E71'}
{'E78', 'E71'}
{'Q12'}
{'E75'}
{'Q80'}
{'Q81'}
{'G60'}
{'G60'}
{'Q78'}
{'Q87'}
{'Q87'}
{'G40'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'G11'}
{'G31'}
{'E70'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'Q21'}
{'D22'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'H35'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q87'}
{'Q61'}

In [22]:
changes

["G71.0 ----> ['G71']",
 "E72.0 ----> ['E72']",
 "G11.8 ----> ['G11']",
 "Q56.1 ----> ['Q56']",
 "H90.3 ----> ['H90']",
 "G31.8 ----> ['G31']",
 "Q80.4 ----> ['Q80']",
 "Q87.0 ----> ['Q87']",
 "Q80.2 ----> ['Q80']",
 "Q87.8 ----> ['Q87']",
 "F42 ----> ['F42']",
 "Q87.8 ----> ['Q87']",
 "G71.0 ----> ['G71']",
 "Q87.8 ----> ['Q87']",
 "Q87.8 ----> ['Q87']",
 "Q77.7 ----> ['Q77']",
 "H90.3 ----> ['H90']",
 "D64.4 ----> ['D64']",
 "E85.0 ----> ['E85']",
 "G12.2 ----> ['G12']",
 "E77.8 ----> ['E77']",
 "Q80.2 ----> ['Q80']",
 "E03.1 ----> ['E03']",
 "Q87.8 ----> ['Q87']",
 "G30 ----> ['G30']",
 "H53.8 ----> ['H53']",
 "Q77.3 ----> ['Q77']",
 "E83.3 ----> ['E83']",
 "Q83.3 ----> ['Q83']",
 "Q87.8 ----> ['Q87']",
 "G11.4 ----> ['G11']",
 "D81.8 ----> ['D81']",
 "G71.0 ----> ['G71']",
 "G11.8 ----> ['G11']",
 "E77.8 ----> ['E77']",
 "Q04.3 ----> ['Q04']",
 "H35.5 ----> ['H35']",
 "C41.9 ----> ['C41']",
 "G11.4 ----> ['G11']",
 "E70.3 ----> ['E70']",
 "D64.0 ----> ['D64']",
 "D18.0 ----> ['D18'

In [26]:
full_ids_mapping

,mondo,omim,snomedct,umls,orpha,mesh,ncit,doid,meddra,medgen,ICD-10
0,0008118,164330,716180009,C1834013,2724,C537740,NaN,NaN,NaN,NaN,NaN
1,0010439,300829,NaN,C1853577,NaN,C543241,NaN,NaN,NaN,NaN,NaN
2,0008117,164310,763829004,C1834014,98897,C563508,NaN,NaN,NaN,NaN,G71.0
3,0009448,242600,84121007,C0268654,42062,C536285,NaN,NaN,NaN,NaN,E72.0
4,0008119,164400,715748006,C0752120,98755,NaN,C129982,0050954,NaN,NaN,G11.8
...,...,...,...,...,...,...,...,...,...,...,...
24115,0009507,245550,732961003,C1855551,1296,C538396,NaN,NaN,NaN,NaN,Q87.8
24116,0009508,245552,NaN,C1855550,NaN,C537549,NaN,NaN,NaN,NaN,NaN
24117,0009501,245340,766715000,C1855577,171690,C565449,NaN,NaN,NaN,NaN,G72.8
24118,0009502,245348,NaN,C1855565,79244,C565448,NaN,NaN,NaN,NaN,E74.4


In [21]:
full_ids_mapping.count()

mondo            24120
omim              8841
snomedct          8962
umls             16234
orpha             9363
mesh              8075
ncit              6953
doid              8944
meddra            1144
medgen               1
ICD-10            9561
parent ICD-10     9561
dtype: int64

In [7]:
def split_and_expand_column(data, split_string, column_name):
    s = data[column_name].str.split(split_string, expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = data.loc[i].copy()
    df2[column_name] = s.values
    return df2

def combine_rows(x):
    return set(filter(None,';'.join(x).split(';')))

def get_disease_mapping(disease_set, id_type):
    # ==== Get Mondo IDs ====
    disease_id_set,_,_ = get_prev_mapping(in_set=disease_set, id_type=id_type, file="../disorders.map", sep="\t")
    mondo_set = list(set('MONDO:'+disease_id_set['mondo']))
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=mondo_set, id_type='mondo', file='disease_disgenet_mapping.csv', sep=",")
    # ==== Get disgenet values ====
    if len(missing) > 0:
        md = get_client("disease")
        mapping = md.getdiseases(missing,
                                 fields='disgenet.genes_related_to_disease.gene_id,disgenet.variants_related_to_disease.rsid,ctd.pathway_related_to_disease.kegg_pathway_id',
                                 species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'mondo'}, inplace = True)
        # transform dataframe to combine single and multiple results
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.genes_related_to_disease', 
                                     singlecol='disgenet.genes_related_to_disease.gene_id', key='gene_id')
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.variants_related_to_disease', 
                                     singlecol='disgenet.variants_related_to_disease.rsid', key='rsid')
        mapping = preprocess_results(mapping=mapping, multicol='ctd.pathway_related_to_disease', 
                                     singlecol='ctd.pathway_related_to_disease.kegg_pathway_id', key='kegg_pathway_id')
        mapping = mapping.drop(columns=['_id','_version','disgenet._license']) 
        #  work with nan float values
        mapping = mapping.fillna('')
        mapping = mapping.astype(str)
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('disease_disgenet_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # ==== Map back to previous ids ====
    df["mondo"] = df["mondo"].str.replace("MONDO:", "")
    # work with not unique values...
    mapping_subset = disease_id_set[['mondo', id_type]].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['mondo'], how = 'outer')
    df = df.drop(columns=['mondo'])
    df = df.fillna('').groupby(id_type, as_index = False).agg({'disgenet.genes_related_to_disease': combine_rows, 'disgenet.variants_related_to_disease': combine_rows, 'ctd.pathway_related_to_disease': combine_rows})
    return df

In [103]:
diseases[0]

0     E10
1     E11
2     E12
3     E13
4     E14
5     E66
6     F00
7     F01
8     F02
9     F03
10    G20
11    G30
12    G43
13    I10
14    I11
15    I12
16    I13
17    I15
18    I21
19    I22
20    I50
21    I63
22    I64
23    I70
24    J45
Name: 0, dtype: object

In [104]:
start = timeit.default_timer()
df = get_disease_mapping(disease_set=diseases[0], id_type='ICD-10')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.16894566499831853


In [105]:
df

,ICD-10,disgenet.genes_related_to_disease,disgenet.variants_related_to_disease,ctd.pathway_related_to_disease
0,E10,"{248, 7225, 177, 241, 5539, 9391, 6925, 407034...","{rs17782313, rs10741657, rs9470794, rs16860234...","{hsa03050, hsa04657, hsa05206, hsa05231, hsa05..."
1,E11,"{248, 7225, 177, 241, 5539, 9391, 6925, 407034...","{rs17782313, rs10741657, rs9470794, rs16860234...","{hsa03050, hsa05220, hsa04657, hsa05206, hsa04..."
2,E13,"{3342, 4540, 2688, 8894, 1729, 3566, 5967, 455...","{rs1135401741, rs121913143, rs111993466, rs797...","{hsa04931, hsa05162, hsa04141, hsa04211, hsa04..."
3,E66,"{248, 8932, 5587, 177, 241, 5321, 5539, 4548, ...","{rs1256046734, rs17782313, rs74421874, rs10182...","{hsa04916, hsa04152, hsa04060, hsa04080, hsa04..."
4,F00,"{6383, 2928, 687, 7225, 57468, 3700, 775, 2365...","{rs165599, rs2535629, rs7224199, rs242941, rs1...",{}
5,F01,"{7422, 1432, 26073, 12, 177, 1401, 5981, 10272...","{rs781563777, rs1555729486, rs28933696, rs8646...","{hsa04371, hsa05224, hsa_M00682, hsa05206, hsa..."
6,F02,{},{},{}
7,G20,"{6908, 7422, 634, 2405, 406901, 51450, 5967, 1...","{rs764786986, rs1272596579, rs63750756, rs2498...","{hsa_M00177, hsa04657, hsa05206, hsa05012, hsa..."
8,G30,"{22953, 248, 8932, 7225, 4300, 6572, 177, 241,...","{rs1925458, rs201825, rs11540652, rs63751235, ...","{hsa04657, hsa05206, hsa04740, hsa04215, hsa05..."
9,G43,"{338567, 8989, 775, 1401, 196740, 4548, 7124, ...","{rs3754701, rs772593652, rs2074193, rs4379368,...","{hsa05224, hsa05220, hsa04390, hsa04917, hsa04..."


# do the comparisson now

### set to set

In [106]:
reference_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,P00325,"{GO:0042573, GO:0001523, GO:0006069, GO:0042572}","{GO:0005829, GO:0005654, GO:0005886}","{GO:0004745, GO:0004024, GO:0008270}","{hsa00010, hsa05204, hsa00620, hsa00071, hsa00..."
1,P00326,"{GO:0042573, GO:0006069, GO:0042572}","{GO:0005829, GO:0005654, GO:0005886}","{GO:0004745, GO:0004024, GO:0004022, GO:0008270}","{hsa00010, hsa05204, hsa00620, hsa00071, hsa00..."
2,P28223,"{GO:0044380, GO:0008219, GO:0014059, GO:004340...","{GO:0030424, GO:0043198, GO:0005901, GO:009905...","{GO:0001618, GO:0071886, GO:0001965, GO:000158...","{hsa04540, hsa04750, hsa04726, hsa04080, hsa04..."
3,P47869,"{GO:0007165, GO:0007214, GO:0007268, GO:000150...","{GO:0030424, GO:0098794, GO:0060077, GO:000588...","{GO:0004890, GO:0022851, GO:1904315, GO:000525...","{hsa04742, hsa05033, hsa04723, hsa04727, hsa04..."


In [107]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0048010, GO:0032869, GO:0051897, GO:005105...","{GO:0005634, GO:0005829, GO:0005942}","{GO:0046935, GO:0001784, GO:0019903, GO:000551...","{hsa05417, hsa05220, hsa05206, hsa05231, hsa05..."
1,O14492,"{GO:0035556, GO:0001922, GO:0019222, GO:005085...","{GO:0001725, GO:0005884, GO:0001726, GO:000582...","{GO:0005068, GO:0042169, GO:0005515, GO:004280...","{hsa04722, hsa04910}"
2,O14610,"{GO:0007602, GO:0007186}",{GO:0005834},"{GO:0003924, GO:0031681}","{hsa05034, hsa04371, hsa04062, hsa04725, hsa05..."
3,O14775,"{GO:0007165, GO:0043547, GO:1901386, GO:000721...","{GO:0098793, GO:0005634, GO:0005829, GO:000573...","{GO:0051087, GO:0030159, GO:0003924, GO:000509...","{hsa05034, hsa04371, hsa04062, hsa04725, hsa05..."
4,O14842,"{GO:0030073, GO:0050796, GO:0032024, GO:003269...","{GO:0005887, GO:0005886}","{GO:0045125, GO:0004930, GO:0008289}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0007156, GO:0016339, GO:0050808, GO:0007155}","{GO:0005887, GO:0016020}",{GO:0005509},{}
196,Q9UNN8,"{GO:0007596, GO:0050819}","{GO:0009986, GO:0005925, GO:0005615, GO:000588...","{GO:0005515, GO:0038023}",{hsa04610}
197,Q9UQC2,"{GO:0051897, GO:0007169, GO:0038095, GO:000741...","{GO:0005829, GO:0005737, GO:0005886}","{GO:0043325, GO:0005068, GO:0005547, GO:0005515}","{hsa05220, hsa04071, hsa04666, hsa04664, hsa04..."
198,Q9Y2G0,"{GO:0072659, GO:0046854}","{GO:0005829, GO:0005886, GO:0015629}",{GO:0005515},{}


In [123]:
reference_dict = dict()
for att_type in reference_kegg_mapping.columns[1:]:
    if att_type not in reference_dict:
        reference_dict[att_type] = set.union(*reference_kegg_mapping[att_type])
reference_dict

{'go.BP': {'GO:0001505',
  'GO:0001523',
  'GO:0001659',
  'GO:0006069',
  'GO:0006836',
  'GO:0006874',
  'GO:0007165',
  'GO:0007186',
  'GO:0007187',
  'GO:0007200',
  'GO:0007202',
  'GO:0007208',
  'GO:0007210',
  'GO:0007214',
  'GO:0007268',
  'GO:0007568',
  'GO:0007613',
  'GO:0008219',
  'GO:0008284',
  'GO:0010513',
  'GO:0014059',
  'GO:0014065',
  'GO:0014824',
  'GO:0014832',
  'GO:0030431',
  'GO:0034220',
  'GO:0042391',
  'GO:0042493',
  'GO:0042572',
  'GO:0042573',
  'GO:0043267',
  'GO:0043406',
  'GO:0044380',
  'GO:0045600',
  'GO:0045821',
  'GO:0045907',
  'GO:0046718',
  'GO:0048148',
  'GO:0050731',
  'GO:0050877',
  'GO:0050965',
  'GO:0050966',
  'GO:0051209',
  'GO:0051932',
  'GO:0051967',
  'GO:0060078',
  'GO:0070374',
  'GO:0098664',
  'GO:1902476',
  'GO:1904862',
  'GO:2000300'},
 'go.CC': {'GO:0005654',
  'GO:0005829',
  'GO:0005886',
  'GO:0005887',
  'GO:0005901',
  'GO:0030285',
  'GO:0030424',
  'GO:0030425',
  'GO:0031410',
  'GO:0032590',
  'GO